In [2]:
### Import the required libraries
import numpy as np
import scipy
import matplotlib.pyplot as plt 

import warnings
warnings.simplefilter("ignore")

import tensorflow as tf
import tensorflow.keras as keras
tf.compat.v1.disable_eager_execution()

import os
import sys

import xarray as xr
import xmitgcm
from xmitgcm import open_mdsdataset

# Append to sys.path the absolute path to src/XAIRT
path_list = os.path.abspath('').split('/')
path_src_XAIRT = ''
for link in path_list[:-1]:
    path_src_XAIRT = path_src_XAIRT+link+'/'
sys.path.append(path_src_XAIRT+'/src')

# Now import module XAIRT
from XAIRT import *

# See if GPUs are available
from keras import backend as K
if bool(K._get_available_gpus()):
    print("Running on GPU")
else:
    print("Running on CPU")

Running on CPU


2023-09-19 00:21:41.970179: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ohpc/pub/libs/gnu7/openmpi/netcdf/4.5.0/lib:/opt/ohpc/pub/libs/gnu7/openmpi/netcdf-fortran/4.4.4/lib:/opt/ohpc/pub/libs/gnu7/openmpi/hdf5/1.10.1/lib:/opt/ohpc/pub/mpi/openmpi-gnu7/1.10.7/lib:/opt/ohpc/pub/compiler/gcc/7.3.0/lib64:/home/shreyas/lis-2.1.3/installation/lib:/share/jdk-16.0.1/lib::
2023-09-19 00:21:41.970260: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-09-19 00:21:41.970326: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c2-1): /proc/driver/nvidia/version does not exist
2023-09-19 00:21:41.971349: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural N

In [19]:
N = 10000
X = np.random.rand(N,2)
y = np.dot(X,np.array([1,2]))[:,np.newaxis]

In [31]:
Layers = [{'size': 2, 'activation': None    , 'use_bias': None},
          {'size': 20, 'activation': 'relu'  , 'use_bias': True},
          {'size': 1 , 'activation': 'linear', 'use_bias': False}]

Losses = [{'kind': 'mse', 'weight': 1.0}]

kwargs = {'losses': Losses, 'optim': 'adam', 'metrics': ['mae'],
            'batch_size': 10, 'epochs': 10, 'validation_split': 0.2,
            'filename': 'model', 'dirname': os.path.abspath('')}

K = TrainFullyConnectedNN(X, y, layers = Layers, **kwargs)
L = TrainLR(X, y, y_ref = 0.0, fit_intercept = False)
best_model = K.quickTrain()
regr = L.quickTrain()
y_NN = best_model.predict(X)
y_LR = regr.predict(X)


Epoch 1: val_loss improved from inf to 0.02762, saving model to /home/shreyas/XAIRT/examples/model.h5

Epoch 2: val_loss improved from 0.02762 to 0.00286, saving model to /home/shreyas/XAIRT/examples/model.h5

Epoch 3: val_loss improved from 0.00286 to 0.00042, saving model to /home/shreyas/XAIRT/examples/model.h5

Epoch 4: val_loss improved from 0.00042 to 0.00025, saving model to /home/shreyas/XAIRT/examples/model.h5

Epoch 5: val_loss improved from 0.00025 to 0.00013, saving model to /home/shreyas/XAIRT/examples/model.h5

Epoch 6: val_loss improved from 0.00013 to 0.00006, saving model to /home/shreyas/XAIRT/examples/model.h5

Epoch 7: val_loss improved from 0.00006 to 0.00003, saving model to /home/shreyas/XAIRT/examples/model.h5

Epoch 8: val_loss improved from 0.00003 to 0.00001, saving model to /home/shreyas/XAIRT/examples/model.h5

Epoch 9: val_loss improved from 0.00001 to 0.00001, saving model to /home/shreyas/XAIRT/examples/model.h5

Epoch 10: val_loss improved from 0.00001

In [32]:
normalizeDict = {'bool_': True, 'kind': 'MaxAbs'}
kwargs = {'y_ref': 0.00}

Xplain = XAIR(best_model, 'lrp.alpha_1_beta_0', 'classic', X, normalizeDict, **kwargs)
a, *_  = Xplain.quick_analyze()
print(np.mean(a, axis = 0))

Xplain = XAIR(best_model, 'gradient', 'classic', X, normalizeDict, **kwargs)
a, *_  = Xplain.quick_analyze()
print(np.mean(a, axis = 0))

Xplain = XLR(regr, X)
a, *_  = Xplain.quick_analyze()
print(np.mean(a, axis = 0))

[0.52979676 0.8786093 ]
[0.50029193 1.        ]
[0.54566141 0.87479875]


In [33]:
regr.coef_

array([[1., 2.]])